In [ ]:
"""
    1. Segmentation result is based on composite/stacking elementary map.
    However, legend and the region outside the EDS mapping area need to be cropped out first
    2. Afterwards, the matching adjustment or alignment is necessary to make sure the segmented coordinates obtained
    from the composite map (multiple elemental maps stacked together) can be used to extract the same grain from the single elementary maps.
"""

In [3]:
import cv2
import os
import numpy as np

# Single Pair Test - with check of size discrepancy

In [ ]:
def generate_image_path_list(input_directory, keyword):
    """
        keyword decides the second image in the list
    
    """
    # Get a list of all files in the input directory
    all_files = os.listdir(input_directory)
    
    # Filter only image files (assuming .tiff, .png, .jpg extensions)
    image_files = [f for f in all_files if f.endswith(('.tiff', '.png', '.jpg', '.jpeg', '.bmp'))]
    
    # Initialize placeholders for the special cases
    elemental_image = None
    keyword_image = None
    other_images = []

    # Loop through image files and classify them
    for image_file in image_files:
        if 'Elemental' in image_file:
            elemental_image = image_file
        elif keyword in image_file:
            keyword_image = image_file
        else:
            other_images.append(image_file)

    # Start building the final image path list
    image_path_list = []

    # Add 'Elemental' image first if it exists
    if elemental_image:
        image_path_list.append(os.path.join(input_directory, elemental_image))

    # Add keyword-based image second if it exists
    if keyword_image:
        image_path_list.append(os.path.join(input_directory, keyword_image))

    # Add the remaining casual images
    for other_image in other_images:
        image_path_list.append(os.path.join(input_directory, other_image))

    # Output and print the final image path list
    print("Generated Image Path List:")
    for path in image_path_list:
        print(path)

    return image_path_list

In [ ]:
'''
    the keyword was 'Si' in this case, so in the generated list, it's the second image. That's just my preference of always comparing with Si map with the composite map first.

'''

In [ ]:
# Example usage:
input_directory = 'To_be_Aligned'  # Specify your input directory
image_paths = generate_image_path_list(input_directory, keyword='Si')


Generated Image Path List:
To_be_Aligned\USU-4183B 150-250 Elemental Map.jpeg
To_be_Aligned\USU-4183B 150-250 Si Map.jpeg
To_be_Aligned\USU-4183B 150-250 Al Map.jpeg
To_be_Aligned\USU-4183B 150-250 Ca Map.jpeg
To_be_Aligned\USU-4183B 150-250 Fe Map.jpeg
To_be_Aligned\USU-4183B 150-250 K Map.jpeg
To_be_Aligned\USU-4183B 150-250 Na Map.jpeg


"\n    the keyward was 'Si' in this case, so in the generated list, it's the second image. That's just my preference of always comparing with Si map with the composite map first.\n\n"

# Single Pair Test

In [4]:
def align_images_with_affine(image1, image2):
    # Convert images to grayscale
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Initialize ORB detector
    orb = cv2.ORB_create(nfeatures=5000)

    # Detect key points and descriptors
    keypoints1, descriptors1 = orb.detectAndCompute(gray1, None)
    keypoints2, descriptors2 = orb.detectAndCompute(gray2, None)

    # Create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Match descriptors
    matches = bf.match(descriptors1, descriptors2)

    # Sort matches by distance and retain the best ones
    matches = sorted(matches, key=lambda x: x.distance)[:30]  # Use the top 30 matches

    # Extract locations of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find the affine transformation (instead of homography) for simplicity in alignment
    affine_matrix, mask = cv2.estimateAffinePartial2D(points2, points1, method=cv2.RANSAC)

    # Use the affine transformation to warp image2 to align with image1
    height, width, channels = image1.shape
    aligned_image2 = cv2.warpAffine(image2, affine_matrix, (width, height))

    return aligned_image2, affine_matrix

In [5]:
# Main function
def main():
    """
    The first in the image list is the composite image;
    the second is the main referenced image.
    """
    # Read the images
    images = [cv2.imread(path) for path in image_paths]
    
    # Use the first two images for initial alignment calculation
    composite_image = images[0]
    imageA = images[1]
    aligned_imageA, affine_matrix = align_images_with_affine(composite_image, imageA)

    # Create output directory named after the composite image under 'Aligned'
    composite_image_name = os.path.basename(image_paths[0]).split('.')[0]
    output_dir = os.path.join('Aligned', composite_image_name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Save the aligned composite image and aligned imageA
    cv2.imwrite(os.path.join(output_dir, f"{composite_image_name}_aligned.png"), composite_image)
    cv2.imwrite(os.path.join(output_dir, f"{os.path.splitext(os.path.basename(image_paths[1]))[0]}_aligned.png"), aligned_imageA)

    # Apply the same affine transformation to the rest of the images and save them
    for image_path in image_paths[2:]:
        image = cv2.imread(image_path)
        aligned_image = cv2.warpAffine(image, affine_matrix, (composite_image.shape[1], composite_image.shape[0]))
        
        aligned_image_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(image_path))[0]}_aligned.png")
        cv2.imwrite(aligned_image_path, aligned_image)

    print(f"All aligned images have been saved in the directory: {output_dir}")

if __name__ == "__main__":
    main()

All aligned images have been saved in the directory: Aligned\USU-4183B 150-250 Elemental Map


**Always manually check the alignment result after running this notebook.**